In [73]:
from bayes_opt import BayesianOptimization
import numpy as np
from plotutil import Figure
import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib inline

plt.rcParams['image.cmap'] = 'jet'

In [74]:
def posterior(bo, x):
    bo.gp.fit(bo.X, bo.Y)
    mu, sigma = bo.gp.predict(x, return_std=True)
    return mu, sigma
    

def plot_gp(bo, x, y=None, axes=None, high_light=False,
            y_lim=(None, None)):
    x = x.reshape(-1, 1)
    
    if axes is None:
        fig = plt.figure(figsize=(6, 4.5))
        fig.suptitle('Gaussian Process and Utility Function After {} Steps'.format(len(bo.X)), fontdict={'size':30})
        
        gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
        axis = plt.subplot(gs[0])
        acq = plt.subplot(gs[1])
    else:
        axis = axes[0]
        acq = axes[1]
    
    mu, sigma = posterior(bo, x)
    if y is not None:
        axis.plot(x, y, linewidth=3, label='Target')
    axis.plot(bo.X.flatten(), bo.Y, 'D', markersize=8, label=u'Observations', color='r')
    if high_light:
        axis.plot(bo.X[-1], bo.Y[-1], 'D', markersize=8, label=u'Observations', color='gold')
    axis.plot(x, mu, '--', color='k', label='Prediction')

    axis.fill(np.concatenate([x, x[::-1]]), 
              np.concatenate([mu - 1.9600 * sigma, (mu + 1.9600 * sigma)[::-1]]),
        alpha=.6, fc='c', ec='None', label='95% confidence interval')
    
    axis.set_xlim((np.min(x), np.max(x)))
    axis.set_ylim(y_lim)
    axis.set_ylabel('S(x)', fontdict={'size':20})
    axis.set_xlabel('x', fontdict={'size':20})
    
    utility = bo.util.utility(x, bo.gp, 0)
    acq.plot(x, utility, label='Utility Function', color='purple')
    acq.plot(x[np.argmax(utility)], np.max(utility), '*', markersize=15, 
             label=u'Next Best Guess', markerfacecolor='gold', markeredgecolor='k', markeredgewidth=1)
    acq.set_xlim((np.min(x), np.max(x)))
    acq.set_ylim((0, np.max(utility) * 1.5))
    acq.set_ylabel('Utility', fontdict={'size':20})
    acq.set_xlabel('x', fontdict={'size':20})
    
    axis.legend(loc=2, bbox_to_anchor=(1.01, 1), borderaxespad=0.)
    acq.legend(loc=2, bbox_to_anchor=(1.01, 1), borderaxespad=0.)

In [75]:
# toy problem 1
x_range = (0.5, 2.5)
grid = np.linspace(*x_range, 100)
Q = np.logical_and(grid >= 1, grid <= 2)

#plt.plot(grid, Q)

In [76]:
# toy problem 2
x_range = (-1.5, 2.5)
grid = np.linspace(*x_range, 100)
Q = np.logical_and(grid >= 1, grid <= 2) + 2*np.logical_and(grid >= -1, grid <= -.5)

In [77]:
def R(x_c, grid=grid):
    return np.exp(-np.abs(grid-x_c))


#plt.plot(grid, R(1.5))

In [78]:
S_1 = lambda x: Q*R(x)


def S(**kwargs):
    x = np.zeros(1)
    x[0] = kwargs['x']
    S_temp = S_1(x[0])
    return np.sum(S_temp.reshape(-1, 1)) / 10

#plt.plot(grid, S_1(x=1.5))

In [79]:
def plot_QS_gp(Q, S_temp, bo, grid, i, y_lim):
    fig = Figure(4, 1, figsize=(16, 12))
    fig[0].plot(grid, Q)
    if i > 0:
        x_last = bo.X[-1]
        fig[1].plot(grid, S_temp(x_last))
    plot_gp(bo, grid, axes=fig[2:4], high_light=i>0, y_lim=y_lim)
    fig[0].set_ylabel('Q(x)', fontdict={'size':20})
    fig[0].set_xlim((np.min(grid), np.max(grid)))
    fig[1].set_ylabel('Q(x)*R(x)', fontdict={'size':20})
    fig[1].set_xlim((np.min(grid), np.max(grid)))
    fig[1].set_ylim((0, 2.5))
    fig.close('figs', f'bo_1d_{i}')


def optimize(bo, init_points, n_iter, y_lim=(None, None)):
    bo.maximize(init_points=init_points, n_iter=0)
    plot_QS_gp(Q, S_1, bo, grid, 0, y_lim=y_lim)
    for i in range(n_iter):
        bo.maximize(n_iter=1)
        plot_QS_gp(Q, S_1, bo, grid, i+1, y_lim=y_lim)

In [80]:
y_lim = (0, 3)
bo = BayesianOptimization(S, {'x': x_range})
optimize(bo, init_points=5, n_iter=10, y_lim=y_lim)

Initialization
-----------------------------------------
 Step |   Time |      Value |         x | 
    1 | 00m00s |    1.98349 |   -1.0610 | 
    2 | 00m00s |    1.73903 |    1.9891 | 
    3 | 00m00s |    1.64414 |    0.2483 | 
    4 | 00m00s |    1.71983 |    0.6461 | 
    5 | 00m00s |    2.19336 |    1.5654 | 


Bayesian Optimization
-----------------------------------------
 Step |   Time |      Value |         x | 


Bayesian Optimization
-----------------------------------------
 Step |   Time |      Value |         x | 


    6 | 00m02s |    1.27877 |   -1.5000 | 


Bayesian Optimization
-----------------------------------------
 Step |   Time |      Value |         x | 


...

   14 | 00m09s |    2.41324 |   -0.7171 | 


/Users/tomokatsu/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -1.16588865e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 60, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


Bayesian Optimization
-----------------------------------------
 Step |   Time |      Value |         x | 


   15 | 00m11s |    2.41259 | 

  -0.6907 | 


In [81]:
#bo = BayesianOptimization(S, {'x': x_range})
#bo.maximize(init_points=3, n_iter=10)
#plot_gp(bo, grid)